# Heatmap of different oscillators

In [1]:
import numpy as np
from simple.plotly import chartFigure
from simple.funcs import symlog, vPIN, cPIN, tickSpeed
from simple.pretty import pmap
from simple.jurik import JRSX, JTPO, JCFB
from simple.ehlers import reflex
from simple.hurst import HurstRS
import matplotlib.pyplot as plt

In [2]:
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.dpi'] = 80
plt.rcParams['figure.subplot.left'] = 0.04
plt.rcParams['figure.subplot.right'] = 0.96
plt.rcParams['figure.subplot.top'] = 0.96
plt.rcParams['figure.subplot.bottom'] = 0.04
plt.rcParams['axes.grid'] = False

In [3]:
T = np.load('data/ETHUSDT.2022-05-08.tick.npz')['ETHUSDT'].view(np.recarray)[:30_000]
len(T)

30000

In [4]:
Buy = T[T.Size > 0]
Sell = T[T.Size < 0]
Buy[-5:]

rec.array([('2022-05-08T00:19:21.746000', 32556, 2630.26, 331.),
           ('2022-05-08T00:19:21.763000', 32557, 2630.26,   2.),
           ('2022-05-08T00:19:21.779999', 32558, 2630.26,   2.),
           ('2022-05-08T00:19:22.347000', 32573, 2630.21,  15.),
           ('2022-05-08T00:19:22.385000', 32584, 2630.13,   2.)],
          dtype={'names':['DT','LocalDT','Price','Size'], 'formats':['<M8[us]','<i8','<f8','<f8'], 'offsets':[0,8,16,24], 'itemsize':40})

In [5]:
# indexed chart without bidask
fig = chartFigure(rows=2, height=500,
    Sell=dict(x=Sell.LocalDT, y=Sell.Price, color='red', mode='markers', size=-symlog(Sell.Size), opacity=0.2),
    Buy=dict(x=Buy.LocalDT, y=Buy.Price, color='green', mode='markers', size=symlog(Buy.Size), opacity=0.2),
    Speed=dict(y=tickSpeed(T, 2000), row=2)
)
fig

FigureWidgetResampler({
    'data': [{'marker': {'color': 'red',
                         'opacity': 0.2,
    …

In [6]:
P = range(3, 600, 3)
k = 120

In [7]:
CPIN = pmap(lambda period: cPIN(T, period*10)[::k], P, postfix='CPIN')

  0%|          | 0/199 [00:00<?, ?it/s]

In [8]:
VPIN = pmap(lambda period: vPIN(T, period*10)[::k], P, postfix='VPIN')
CPIN = pmap(lambda period: cPIN(T, period*10)[::k], P)
RSX = pmap(lambda period: JRSX(T.Price, period*10)[::k], P)
CFB = pmap(lambda period: JCFB(T.Price, 7, period*10)[::k], P)
RF = pmap(lambda period: reflex(T.Price, period*10)[::k], P)
SP = pmap(lambda period: tickSpeed(T, period*10)[::k], P)

  0%|          | 0/199 [00:00<?, ?it/s]

  0%|          | 0/199 [00:00<?, ?it/s]

  0%|          | 0/199 [00:00<?, ?it/s]

  0%|          | 0/199 [00:00<?, ?it/s]

  0%|          | 0/199 [00:00<?, ?it/s]

  0%|          | 0/199 [00:00<?, ?it/s]

In [9]:
HURST = pmap(lambda period: HurstRS(np.diff(T.Price), 600 + period*10)[::k], P)

  0%|          | 0/199 [00:00<?, ?it/s]

In [ ]:
TPO = pmap(lambda period: JTPO(T.Price, period*10)[::k], P)

  0%|          | 0/199 [00:00<?, ?it/s]

In [ ]:
N = 8

plt.rcParams["image.aspect"] = 'auto'

fig, ax = plt.subplots(N, 1, figsize=(16, 13), sharex='col')
fig.subplots_adjust(hspace=0)
ax[0].imshow(VPIN, cmap='RdYlGn', vmin=-100, vmax=100)
ax[0].set_title('vPIN', y=0.8)

ax[1].imshow(CPIN, cmap='RdYlGn', vmin=-100, vmax=100)
ax[1].set_title('cPIN', y=0.8)

ax[2].imshow(RSX, cmap='RdYlGn', vmin=0, vmax=100)
ax[2].set_title('RSX', y=0.8)

ax[3].imshow(CFB, cmap='RdYlGn', vmin=0, vmax=200)
ax[3].set_title('CFB', y=0.8)

ax[4].imshow(RF, cmap='RdYlGn', vmin=-2, vmax=2)
ax[4].set_title('Reflex', y=0.8)

ax[5].imshow(TPO, cmap='RdYlGn', vmin=-1.5, vmax=1.5)
ax[5].set_title('TPO', y=0.8)

ax[6].imshow(SP, cmap='RdYlGn', vmin=-0.75, vmax=0.75)
ax[6].set_title('Speed', y=0.8)

ax[7].imshow(HURST, cmap='RdYlGn', vmin=0, vmax=1.3)
ax[7].set_title('Hurst R/S', y=0.8)

[ax[t].twinx().plot(T.Price[::k]) for t in range(N)];